# Simple Multi-Period Portfolio Optimization Example

## 1. Setup

#### 1.1 Load InvestOS module

In [1]:
# Add relative path to module lookup path...
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

# ... then import module
import invest_os as inv

# Import other required modules
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

#### 1.2 Load historical stock prices and volumes

Place in examples/data directory [data from Kaggle](https://www.kaggle.com/datasets/camnugent/sandp500/download?datasetVersionNumber=4)

Note: you will need to create (and sign into) a free Kaggle account

In [2]:
# A. Load S&P 500 tickers, names, and industries
dir_name = './data/'

df = pd.read_csv(
    dir_name + 'all_stocks_5yr.csv'
).rename(columns={'Name': 'asset'})

In [3]:
# B. Keep date, price (mid-way between bid and ask), and volume only
df = df[['asset', 'date', 'open', 'volume']]
df = df.rename(columns={'open': 'price'})

#### 1.3 Create (fake) historical stock bid/ask spreads

In [4]:
# Note: real data should be used for this, wherever possible
# For now, assume 0.01% (i.e. 1 cent on $100.00 stock)
df['spread'] = 1/10_000

#### 1.4 Clean DF

In [5]:
# Keep stocks (assets) with less than 10 price nulls only
df = df.groupby('asset').filter(lambda g: g['price'].isnull().sum().sum() < 10)
# Make date type datetime
df['date'] = pd.to_datetime(df['date'])

#### 1.5 Calculate returns

In [6]:
df['price_t+1'] = df.groupby(['asset'])['price'].shift(-1)

In [7]:
df['return'] = df['price'] / df['price_t+1'] - 1 # fwd looking return

In [8]:
# Keep required columns only
df = df[['asset', 'date', 'price', 'return', 'volume', 'spread']]

# Drop null returns
df = df[df['return'].notnull()]

In [9]:
# Drop where return data insufficient
UNIQUE_RETURNS_P_THRESHOLD = 0.99

s_nunique = df.groupby(['asset']).nunique()['return']
max_unique_returns = s_nunique.max()
df_percent_returns_unique = (s_nunique / max_unique_returns).reset_index()
df_percent_returns_unique = df_percent_returns_unique.rename(columns={'return': 'p_unique'})
df = df.merge(df_percent_returns_unique, on=['asset'], how='left')
df = df[df['p_unique'] >= UNIQUE_RETURNS_P_THRESHOLD]
df = df.drop(columns=['p_unique'])

#### 1.6 Split dfs into historical and forecast

In [10]:
date_split_forecasts = '2016-01-01'

df_historical = df[df.date < date_split_forecasts]
df_forecast = df[df.date >= date_split_forecasts][['asset', 'date', 'return']]
df_actual = df_forecast.copy()

#### 1.7 Create (fake) forecasts

In [11]:
np.random.seed(0)

# Median return should be VERY close to 0...
print("Median return:", df_forecast['return'].median())

# ... so divide by 20x to reduce signal...
df_forecast['return'] /= 20

# ... then add guassian noise to daily returns
std = df_historical['return'].var() ** 0.5
noise = np.random.normal(0, std, size=df_forecast.shape[0])

df_forecast['return'] = df_forecast['return'] + noise

Median return: -0.0008741769337591299


In [12]:
# Make sure predictions aren't too accurate:

agree_on_sign = np.sign(df_forecast['return']) == np.sign(df_actual['return'])

print(
    "Return predictions have the right sign %.1f%% of the time" %
    ((agree_on_sign.sum() / agree_on_sign.shape[0]) * 100)
)

Return predictions have the right sign 51.1% of the time


## 2. Portfolio optimization

#### 2.1 Create portfolio optimization instance

In [13]:
strategy = inv.portfolio_optimization.strategy.RankLongShort(
    n_periods_held = 1,
    leverage = 1
)

po = inv.portfolio_optimization.Optimizer(
    df_forecast,
    df_actual,
    df_historical,
    strategy=strategy
)

In [14]:
backtest = po.optimize()

Optimizing...
Done simulating.


In [15]:
# po.backtest.h["cash"][165:200]

# po.backtest.h["AAPL"][-20:]

# po.backtest.h

# po.backtest.h[np.abs(po.backtest.h) < 0.001]

po.backtest.v

# po.backtest.u[po.backtest.u.index != "cash"].iloc[5, :]

2016-01-04    1.000000e+08
2016-01-05    1.000553e+08
2016-01-06    9.998904e+07
2016-01-07    9.986177e+07
2016-01-08    9.976282e+07
                  ...     
2018-01-31    1.455942e+08
2018-02-01    1.460500e+08
2018-02-02    1.461240e+08
2018-02-05    1.461915e+08
2018-02-06    1.467001e+08
Length: 528, dtype: float64